In [ ]:
def generate_heatmap_of_tile(model, tilepath, composite, device="cpu"):
    # load tile
    
    # 

In [ ]:
models = []
tile = "path_to_tile"
# plot original


for m in models:
    hm = generate_heatmap_of_tile(tile)
    plot(hm)
    print(m.name)
    print(stats)